# Chabot using Llama 3.2-1B (Large Language Model)

The Llama 3.2 collection of multilingual large language models (LLMs) is a collection of pretrained and instruction-tuned generative models in 1B and 3B sizes (text in/text out). The Llama 3.2 instruction-tuned text only models are optimized for multilingual dialogue use cases, including agentic retrieval and summarization tasks. They outperform many of the available open source and closed chat models on common industry benchmarks.

**Supported Languages:** English, German, French, Italian, Portuguese, Hindi, Spanish, and Thai are officially supported. Llama 3.2 has been trained on a broader collection of languages than these 8 supported languages. Developers may fine-tune Llama 3.2 models for languages beyond these supported languages, provided they comply with the Llama 3.2 Community License and the Acceptable Use Policy. Developers are always expected to ensure that their deployments, including those that involve additional languages, are completed safely and responsibly.

**Model Release Date:** Sept 25, 2024

**License:** Use of Llama 3.2 is governed by the Llama 3.2 Community License (a custom, commercial license agreement).

In [1]:
pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 79.4 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.1
    Uninstalling transformers-4.47.1:
      Successfully uninstalled transformers-4.47.1


In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineG

In [4]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="meta-llama/Llama-3.2-1B")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

Device set to use cuda:0


In [5]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B")

In [6]:
import pandas as pd
data = pd.read_excel('/content/drive/MyDrive/gpt-4.xlsx')

In [7]:
import re
import nltk
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from collections import Counter

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')  # This line downloads the necessary data

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [8]:
# Predefined stopwords
stop_words = set(stopwords.words('english'))

# Function to preprocess text
def preprocess_text(text):
    if isinstance(text, str):  # Check if the value is a string
        # Remove special characters
        text = re.sub(r'[^a-zA-Z0-9\s-]+', '', text)
        # Remove stopwords
        text = " ".join([word for word in text.split() if word.lower() not in stop_words])
        # Remove additional spaces
        text = re.sub(r'\s+', ' ', text).strip()
    return text

# Apply preprocessing to both columns
columns_to_process = ['data', 'conversation']
for column in columns_to_process:
    data[f'processed_{column}'] = data[column].apply(preprocess_text)

In [9]:
data = data.drop(columns=['data', 'conversation'])

In [10]:
from sentence_transformers import SentenceTransformer, util

# Load Sentence Transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Define the input and response columns
inputs = data['processed_data']
responses = data['processed_conversation']

# Encode responses
response_embeddings = model.encode(responses, convert_to_tensor=True)

# Function to retrieve the best response
def get_response(user_input):
    query_embedding = model.encode(user_input, convert_to_tensor=True)
    scores = util.pytorch_cos_sim(query_embedding, response_embeddings)
    best_match_idx = scores.argmax().item()
    return responses.iloc[best_match_idx]

# Example queries
print(get_response('Tell me about COVID19 patients?'))

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Doctor Hi feeling today Patient Im feeling well doctor high fever pain abdomen Doctor Okay tell symptoms experiencing vomiting diarrhea Patient Yes vomiting diarrhea past two days Doctor noticed blood vomit stool Patient Yes two episodes melena hematemesis Doctor see contact COVID-19 positive patient recently Patient Yes contact positive patient month ago didnt symptoms Doctor Okay see Based symptoms history possible dengue fever need tests confirm Patient Okay kind tests Doctor need check blood pressure platelet count lab tests check signs infection may also need chest x-ray echocardiogram check heart lungs Patient Alright treatment need Doctor dengue fever need give fluids inotropes help plasma leakage also give furosemide help swelling levofloxacin treat infections need receive oxygen via face mask help breathing Patient Okay thank doctor Doctor also need monitor platelet count lab values closely make sure youre improving need may need give blood transfusion Patient chances recovery

In [11]:
get_response('What are the symptoms of COVID19 patient?')

'Doctor Good afternoon help today Patient Hi doctor Ive experiencing persistent forehead swelling past two months Doctor Could tell little bit past medical history medical conditions medications youre taking Patient Yes hypertension recently recovered COVID-19 infection took dexamethasone Doctor see take antibiotics time Patient Yes Im sure names courses Doctor Okay thank Based symptoms possible may developed sinusitis Well need evaluation confirm Patient Alright kind evaluation need Doctor Well need CT scan head see whats going Depending results may need start course antibiotics Patient Okay sounds good possible side effects antibiotics Doctor common side effects nausea diarrhea stomach pain However important take full course antibiotics prescribed ensure infection fully treated Patient Understood happens antibiotics dont work Doctor antibiotics dont work may need consider surgery remove infected tissue However well cross bridge come Patient Alright thank explaining everything Doctor 

In [12]:
get_response("Can you tell me about fever?")

'Doctor Hello feeling today Patient Im feeling well Ive fever Doctor see referred us evaluation fever unknown origin tell fever started high got Patient started week ago got 40C Doctor experienced symptoms coughing vomiting diarrhea Patient havent symptoms Doctor Okay see previous doctor treated first-generation cephalosporin didnt lead reduction fever correct Patient Yes thats right Doctor noticed skin rashes weight loss Patient havent noticed anything like Doctor Alright Well need physical examination get information noticed abnormalities body Patient Ive feeling pale febrile Doctor Okay see physical examination noted splenomegaly confirmed ultrasound showed spleen size 99 cm Thats larger normal age Patient Oh didnt know Doctor also found abnormalities blood tests haemoglobin level 75 gl platelet count 50 109l also noticed 175 suspected atypical cells C-reactive protein level 73 mgl erythrocyte sedimentation rate 47 mmh liver function tests abnormal aspartate aminotransferase ASAT 14

In [13]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import numpy as np
import torch

def make_batches(data, batch_size=32):
    """
    Creates batches of data using a generator to avoid loading the entire dataset into memory.
    """
    num_samples = len(data)
    for i in range(0, num_samples, batch_size):
        yield data[i : min(i + batch_size, num_samples)]

# Check if GPU is available and set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the model and tokenizer, and move them to the device
model_name = "meta-llama/Llama-3.2-1B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=device)

def find_most_relevant_context(user_query, data):
    """
    Finds the most relevant context from the dataset based on the user query.

    This is a simple example using keyword matching.
    """
    keywords = user_query.lower().split()
    most_relevant_index = 0
    max_keyword_matches = 0

    for index, row in data.iterrows():
        processed_data = row['processed_data'].lower()
        matches = sum(keyword in processed_data for keyword in keywords)
        if matches > max_keyword_matches:
            max_keyword_matches = matches
            most_relevant_index = index

    return most_relevant_index

# Interactive chatbot loop
print("Chatbot is ready! Ask your questions (type 'exit' to quit).")
while True:
    user_query = input("\nYou: ")
    if user_query.lower() == "exit":
        print("Chatbot session ended. Goodbye!")
        break

    # Find the most relevant context
    most_relevant_index = find_most_relevant_context(user_query, data)
    processed_data = data.loc[most_relevant_index, 'processed_data']
    processed_conversation = data.loc[most_relevant_index, 'processed_conversation']

    # Prepare input for the model with the selected context
    prompt = f"Context Data: {processed_data}\nPrevious Conversation: {processed_conversation}\nUser Query: {user_query}\nResponse:"
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # Generate response with Llama 3.2
    outputs = model.generate(**inputs, max_new_tokens=100)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Print the response
    print(f"Chatbot: {generated_text}")

Device set to use cuda


Chatbot is ready! Ask your questions (type 'exit' to quit).

You: What medication can be given to patient who is suffering from fever.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Chatbot: Context Data: 58-year-old Caucasian American male unremarkable past medical history presented evaluation nausea vomiting 30-pound weight loss past two months hospital also reported dark-colored urine intermittent episodes hemoptysis period Specifically stated symptoms started four days receiving second dose mRNA-1273 Moderna vaccine COVID-19 first dose taken three weeks earlier well tolerated denied flank abdominal pain melena fever cough hematuria urinary frequency urgency trauma denied smoking Vital signs stable upon admission Physical examination insignificant lower extremity pitting edema petechiae rash patient medication prior hospitalization Laboratory analysis remarkable serum creatinine 41 mgdL 08-14 mgdL along hematuria sub-nephrotic proteinuria 1796 g24 hours 150 mg24 hours differential diagnosis point wide including nephritic syndromes given AKI hematuria proteinuria serological workup subsequently sent C-ANCA anti-neutrophil cytoplasmic antibodies elevated 160 AUmL

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Chatbot: Context Data: 25-year-old woman admitted due complaining difficulty swallowing youngest five siblings also non-smoker non-drinker child consanguineous marriage father mother cousins Furthermore five years old diagnosed Fanconi anemia confirm diagnosis FA communicated related university hospital records patients 20 years ago approved diagnosis FA peripheric blood sample studies without giving details offered bone marrow transplantation parents approved major symptoms 20 years disease control patient go regular hospital check-ups patients first notable characteristic growth retardation manifested short stature microcephaly microphthalmia Afterward swallowing difficulty last two years lost around 10 kg last six months patient referred gastroenterology department due swallowing problems endoscopic examination stenosis observed hypopharynx restricts passage scope follows Positron emission tomography PET-CT scan showed multiple lymphadenopathies bilateral deep cervical lymph nodes s

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Chatbot: Context Data: 64-year-old Caucasian male smoker horseshoe kidney history open pyelolithotomy 18 years ago presented King Abdulaziz Medical City mid-2020 report another hospital stating developed gross hematuria six months prior treated urinary tract infection CT abdomen pelvis performed hospital showing horseshoe kidney severe left hydronephrosis enlarged retroperitoneal lymph nodes largest one located posterior part left renal artery measuring 47 35 26 cm Additionally multiple stones Figures Urine culture performed revealed various organisms isolated 10-100000 CFUml Urinalysis showed small amount blood moderate presence leukocytes trace protein end 2020 patient underwent magnetic resonance imaging MRI MRI showed horseshoe kidney chronic hydronephrosis left kidney large mass within centrally satellite lesions likely represent UC associated lymphadenopathy along para-aortic chain Figure Additionally finding chronic pancreatitis noted dilated duct stone patient referred gastroen